# Use AutoGen together with MemGPT

This notebook is run in an environment where AutoGen and MemGPT have been properly installed, following the instructions from MemGPT.

Disclaimer: Code in this notebook is adopted from a notebook in MemGPT's offical github repo. 


In [3]:
import openai
openai.api_key=open("key.txt").read().strip()

In [5]:
"""Example of how to add MemGPT into an AutoGen groupchat
"""

import os
import autogen
from memgpt.autogen.memgpt_agent import create_autogen_memgpt_agent

config_list = [
    {
        "model": "gpt-4",
        "api_key": os.getenv("OPENAI_API_KEY"),
    },
]

# If USE_MEMGPT is False, then this example will be the same as the vanilla AutoGen.
# If USE_MEMGPT is True, then we swap out the "coder" agent with a MemGPT agent.
USE_MEMGPT = True
# If DEBUG is False, a lot of MemGPT's inner workings output is suppressed and only the final send_message is displayed.
# If DEBUG is True, then all of MemGPT's inner workings (function calls, etc.) will be output.
DEBUG = True

llm_config = {"config_list": config_list, "seed": 42}

# The user agent
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
    human_input_mode="TERMINATE",  # needed?
)

# The agent playing the role of the product manager (PM)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)

if not USE_MEMGPT:
    # In the AutoGen example, we create an AssistantAgent to play the role of the coder
    coder = autogen.AssistantAgent(
        name="Coder",
        llm_config=llm_config,
    )

else:
    # In our example, we swap this AutoGen agent with a MemGPT agent
    # This MemGPT agent will have all the benefits of MemGPT, ie persistent memory, etc.
    coder = create_autogen_memgpt_agent(
        "MemGPT_coder",
        persona_description="I am a 10x engineer, trained in Python. I was the first engineer at Uber (which I make sure to tell everyone I work with).",
        user_description=f"You are participating in a group chat with a user ({user_proxy.name}) and a product manager ({pm.name}).",
        interface_kwargs={"debug": DEBUG},
    )

# Initialize the group chat between the user and two LLM agents (PM and coder)
groupchat = autogen.GroupChat(agents=[user_proxy, pm, coder], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# Begin the group chat with a message from the user
user_proxy.initiate_chat(
    manager,
    message="I want to design an app to make me one million dollars in one month. Yes, your heard that right.",
)

LLM is explicitly disabled. Using MockLLM.
User_proxy (to

 chat_manager):

I want to design an app to make me one million dollars in one month. Yes, your heard that right.

--------------------------------------------------------------------------------
Product_manager (to chat_manager):

Given your ambitious goal, here are a few innovative app ideas:

1. "Luxury Concierge" - This app caters to ultra-rich clients who seek unique experiences and products. Whether it's booking private jets, reservations at world-class restaurants, or exclusive tickets to events, this app would be a one-stop solution for luxurious requests. Include a premium subscription for personalized services.

2. "Elite Auction Access" - An app offering access to global auctions for fine art, antiquities, rare cars, yachts, etc. The app can earn by getting a commission per transaction.

3. "Investment Guru" - A platform where top investors share their tips and analysis, users pay to subscribe to these top investors, giving them a direct line to financial advice.

4. "Blockc